In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv("../.env")



True

In [2]:
from typing import Annotated

from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import BaseTool, tool
from langchain_core.runnables import Runnable

def create_agent(llm: ChatOpenAI, tools, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    agent = create_react_agent(llm, tools, state_modifier=prompt)
    return agent

In [4]:
# from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.tools import Tool
# from langchain.chat_models import ChatOpenAI

# def supervisor_tool(messages):
#     members = ["Researcher", "Coder"]
#     system_prompt = (
#         "You are a supervisor tasked with managing a conversation between the"
#         " following workers:  {members}. Given the following user request,"
#         " respond with the worker to act next. Each worker will perform a"
#         " task and respond with their results and status. When finished,"
#         " respond with FINISH."
#     )
#     options = ["FINISH"] + members
#     function_def = {
#         "name": "route",
#         "description": "Select the next role.",
#         "parameters": {
#             "title": "routeSchema",
#             "type": "object",
#             "properties": {
#                 "next": {
#                     "title": "Next",
#                     "anyOf": [
#                         {"enum": options},
#                     ],
#                 }
#             },
#             "required": ["next"],
#         },
#     }
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", system_prompt),
#             MessagesPlaceholder(variable_name="messages"),
#             (
#                 "system",
#                 "Given the conversation above, who should act next?"
#                 " Or should we FINISH? Select one of: {options}",
#             ),
#         ]
#     ).partial(options=str(options), members=", ".join(members))

#     llm = ChatOpenAI(model="gpt-4o-mini")

#     supervisor_chain = (
#         prompt
#         | llm.bind_functions(functions=[function_def], function_call="route")
#         | JsonOutputFunctionsParser()
#     )

#     # Execute the supervisor chain with the provided messages
#     result = supervisor_chain.invoke({"messages": messages})
#     return result

# # Define the function as a tool
# supervisor_tool_tool = Tool(
#     name="supervisor_tool",
#     func=supervisor_tool,
#     description="A tool that acts as a supervisor to manage a conversation between workers and decides the next action."
# )


In [5]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser

members = ["Instructor", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4o-mini")

supervisor_chain = (
    prompt
    | llm.bind_tools(tools=[function_def], tool_choice="required")
)

def supervisor_agent(state):
    response = supervisor_chain.invoke(state)
    answer = response.tool_calls[0]["args"]["next"]
    state["next"] = answer
    print(f"Next: {answer}")
    return state

In [57]:
# Define code tools

@tool
def swap_scramble_row(index1, index2):
    """Swap two rows in the scramble dataframe. Return the new dataframe as string.
    Args:
        index1: The index of the first row to swap.
        index2: The index of the second row to swap.
    """
    scramble.iloc[index1], scramble.iloc[index2] = scramble.iloc[index2].copy(), scramble.iloc[index1].copy()
    return scramble.to_string()

@tool
def swap_scramble_column(column1, column2):
    """Swap two columns in the scramble dataframe. Return the new dataframe as string.
    Args:
        column1: The name of the first column to swap.
        column2: The name of the second column to swap.
    """
    scramble[column1], scramble[column2] = scramble[column2].copy(), scramble[column1].copy()
    return scramble.to_string()

@tool
def change_scramble_column(column, new_column):
    """Change the name of a column in the scramble dataframe. Return the new dataframe as string.
    Args:
        column: The name of the column to change.
        new_column: The new name for the column.
    """
    scramble.rename(columns = {column : new_column}, inplace = True)
    return scramble.to_string()

@tool
def change_scramble_row(old_name, new_name):
    """Change the name of a row in the scramble dataframe. Return the new dataframe as string.
    Args:
        old_name: The index of the row to change.
        new_name: The new name for the row.
    """
    scramble.rename(index = {old_name : new_name}, inplace = True)
    return scramble.to_string()

@tool
def get_truth():
    """get the truth dataframe"""
    return truth.to_string()

@tool
def get_scramble():
    """get the scramble dataframe"""
    return scramble.to_string()

In [58]:
import functools
import operator
from typing import Sequence, TypedDict

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # The 'next' field indicates where to route to next
    next: str





In [59]:
instructor_agent = create_agent(llm, [get_scramble, get_truth], "Your job is the check scramble and truth to make sure that scramble is as close to truth as possible. If not, ask the coder to make specific changes, like swapping columns or changing the name of columns/rows. BE specific. Do NOT make any modifications yourself.")


In [60]:
# python_repl_tool, 
code_agent = create_agent(
    llm,
    [change_scramble_column, swap_scramble_column, swap_scramble_row, change_scramble_row],
    "You specialize in modifying scramble. You are the heavy lifter of the team and can carry out the modifications needed with the instructions given by instructor. When done with your job, say you are done. ",
)

In [61]:
workflow = StateGraph(AgentState)
workflow.add_node("Instructor", instructor_agent)
workflow.add_node("Coder", code_agent)
workflow.add_node("supervisor", supervisor_agent)

In [62]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

In [63]:
import pandas as pd
scramble = pd.read_csv("scrambled.csv")
truth = pd.read_csv("truth.csv")

scramble_string = scramble.to_string()
truth_string = truth.to_string()
scramble_string
temp = "I have changed two dataframes, called scramble and truth, to strings. I want you to convert them back to dataframes "
"and them change scramble to resemble truth as closely as possible without changing the quantitative information within. Do not "
"change truth, only scramble. The two dataframes are truth and scramble as follows: " + scramble_string + " and " + truth_string + "."
temp2 = "Given a string of a dataframe, convert it back into a dataframe and them generate another row before returning the string representation of your new dataframe. Use this string: "+ scramble_string + "."

In [64]:
from langchain_core.messages import HumanMessage
input  ={
        "messages": [
            HumanMessage(content="I have changed two dataframes, called scramble and truth, to strings. I want you to convert them back to dataframes "
"and them change scramble to resemble truth as closely as possible without changing the quantitative information within. Instructor can give you both scramble and truth as strings."),
        ]
    }
for output in graph.stream(input, stream_mode="values"):
    # stream() yields dictionaries with output keyed by node name
    messages = output["messages"]
    for message in messages:
        message.pretty_print()
    print("\n---\n")

================================ Human Message =================================

I have changed two dataframes, called scramble and truth, to strings. I want you to convert them back to dataframes and them change scramble to resemble truth as closely as possible without changing the quantitative information within. Instructor can give you both scramble and truth as strings.

---

Next: Instructor
================================ Human Message =================================

I have changed two dataframes, called scramble and truth, to strings. I want you to convert them back to dataframes and them change scramble to resemble truth as closely as possible without changing the quantitative information within. Instructor can give you both scramble and truth as strings.

---

================================== Ai Message ==================================

Please provide the strings for both the `scramble` and `truth` dataframes, and I'll assist you in converting them back to dataframes 

In [67]:
scramble

,Sales,Profit,Units Sold
Product 1,101,3041,71
Product 2,655,4499,50
Product 3,665,4417,94
Product 4,205,3612,89
Product 5,971,3061,87
Product 6,489,3824,49
Product 7,370,2955,80
Product 8,962,3253,74
Product 9,576,3139,33
Product 10,829,2478,69
